<a href="https://colab.research.google.com/github/likeaAI/ai_study/blob/main/220602/1_13_6_keras_stock_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##LSTM에 맞는 데이터셋 준비하기
yahoo에서 코스피 지수 데이터가져와서, 차트 분석한다

###<Yahoo Finace 주가데이터 사용상 주의>
yahoo사이트의 yfinance 데이터 가져오는 방법은 수십종류가 있고,수시로 변경하거나 업그레이드되므로 사용에 주의 요망

pandas_datareader.data(단축어 pdr)를 사용하여 데이터 다운받으려고 하면,pdr의.get_data_yahoo() 함수를 사용하기위해 yf.pdr_override()를 붙여야한다

In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
import yfinance as yf
import pandas_datareader.data as pdr

yf.pdr_override()
df_prices=pdr.get_data_yahoo("^KS11", start="2020-01-01", end="2021-07-30") 
prices=df_prices['Close']

[*********************100%***********************]  1 of 1 completed


In [ ]:
df_prices

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,2201.209961,2202.320068,2171.840088,2175.169922,2175.169922,494700
2020-01-03,2192.580078,2203.379883,2165.389893,2176.459961,2176.459961,631600
2020-01-06,2154.969971,2164.419922,2149.949951,2155.070068,2155.070068,592700
2020-01-07,2166.600098,2181.620117,2164.270020,2175.540039,2175.540039,568200
2020-01-08,2156.270020,2162.320068,2137.719971,2151.310059,2151.310059,913800
...,...,...,...,...,...,...
2021-06-29,3305.260010,3305.419922,3276.629883,3286.679932,3286.679932,1211800
2021-06-30,3288.629883,3305.729980,3288.629883,3296.679932,3296.679932,1608900
2021-07-01,3297.919922,3300.120117,3278.100098,3282.060059,3282.060059,1157700


In [ ]:
import numpy as np
from keras.models import *
from keras.layers import * 

예측하는 다음날까지 겹치는 윈도우 window_data만들기

In [ ]:
sequence_length = 31
window_data = []
for index in range(len(prices) - sequence_length):
        window_data.append(prices[index: index + sequence_length])  

deep learning 모델의 트레이닝과 테스트로 사용되는 데이터는 numpy 배열로 준비

In [ ]:
normalised_data = []
for window in window_data:
    normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
    normalised_data.append(normalised_window)
    result = np.array(normalised_data)

In [ ]:
result.shape, result[:,-1].shape

((308, 31), (308,))

##LSTM 훈련 데이터의 형식은 3차원으로 데이터 전처리한다
LSTM 입력데이터는  [batch_size, timesteps, feature] 3차원으로 구성되어 있다.일반 신경망모델의 입력데이터는 size와 feature만 있지만, 시간 개념timestep을 넣어 차원을 하나 늘린 것이다.

당초 데이터 result의 차원이 2차원으로 size는 날짜에 따라 변동, timestep은 31으로 ((308, 31), (308,)) 값으로 만든다. 여기에서 3차원으로 timestep은 30으로, feature는 1을 늘리고 train과 test로 데이터를 분할한다 ((277, 30, 1), (277,), (31, 30, 1), (31,))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(result,result[:,-1],test_size=0.1,random_state=0)

In [ ]:
x_train = x_train[:,:-1,None]
x_test =  x_test[:,:-1,None]

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((277, 30, 1), (277,), (31, 30, 1), (31,))

##LSTM 신경망모델 학습과정구성,훈련반복시키기

input_shape=(timestep, feature)을 모델에 알려주고,예측하고자하는 target의 갯수가 1이므로 마지막 층 Dense 노드는 1이다. 

[출력층 활성화함수는 분류에는 softmax, 특히 2진 분류는 sigmoid사용한다. 회귀에는 linear 등 항등함수 사용하며 생략하기도 한다]

본 신경망은 Dense출력층 노드가 1개로 이곳으로 활성화함수 linear를 통해 출력은 회귀 예측값이 된다.

연속적인 값을 예측위한 손실함수는 mean squared error, optimizer는 rmsprop 사용

In [ ]:
model = Sequential()
model.add(LSTM(units=30, return_sequences=True, input_shape=(30, 1))) 
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')  
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 30)            3840      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24320     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 28,225
Trainable params: 28,225
Non-trainable params: 0
_________________________________________________________________


###LSTM 매개변수 parameter 계산
Dense와 다르게 LSTM은 총계산을 한번에 한다.첫번째 LSTM parameter = 4 × ((x + h) × h + h)으로,4*(1+30)*30+30 = 3840 이다. 두번째 LSTM의 parameter는 4*((30+64)*64+64) = 24320이다. 세 번째 Dense층은 출력층을 겸하며, 파라미터는 64(입력 유닛수) x 1(두 번째 Dense층 유닛수,out 크기) + 1(유닛별로 가진 편향수) = 65이다

여기서 4 functions은 LSTM속에 있는 연산함수 숫자이고, 입력은 3차원 특성의 x , 다음 레이어로 벡터 출력 공간의 차원 h이다

In [ ]:
model.fit(x_train, y_train,batch_size=15,epochs=20)

Epoch 1/20
19/19 [==============================] - 4s 24ms/step - loss: 0.0053
Epoch 2/20
19/19 [==============================] - 0s 24ms/step - loss: 0.0015
Epoch 3/20
19/19 [==============================] - 0s 23ms/step - loss: 0.0012
Epoch 4/20
19/19 [==============================] - 0s 24ms/step - loss: 0.0013
Epoch 5/20
19/19 [==============================] - 0s 24ms/step - loss: 8.3304e-04
Epoch 6/20
19/19 [==============================] - 0s 24ms/step - loss: 8.4229e-04
Epoch 7/20
19/19 [==============================] - 1s 27ms/step - loss: 0.0011
Epoch 8/20
19/19 [==============================] - 1s 29ms/step - loss: 7.4746e-04
Epoch 9/20
19/19 [==============================] - 1s 29ms/step - loss: 8.3704e-04
Epoch 10/20
19/19 [==============================] - 1s 28ms/step - loss: 8.0477e-04
Epoch 11/20
19/19 [==============================] - 1s 28ms/step - loss: 7.4293e-04
Epoch 12/20
19/19 [==============================] - 1s 29ms/step - loss: 6.2941e-04
Epoch 13/

예측치와 실제주가 변동이 비슷하다

In [ ]:
pred = model.predict(x_test)[:,-1]
import plotly.graph_objects as go     
fig = go.Figure()   
fig.add_trace(go.Scatter(y=y_test,mode='lines',hovertext='실제값'))
fig.add_trace(go.Scatter(y=pred,mode='lines+markers',hovertext='예측치'))

###<책의 pandas_datareader오류>
 pandas_datareader 라이브러리로 yahoo에서 직접 주가 가져오는 api기능은 URL 읽지못하는 오류발생해서 2021.7.5일부터 사용하지 못함 

In [ ]:
#from pandas_datareader import get_data_yahoo
#import pandas as pd
#df_prices=pd.DataFrame(get_data_yahoo('^KS11', '2020-1-01')) 
#prices = df_prices['Close']